# Gensim 패키지
- Python으로 작성된 오픈 소스 라이브러리로, 자연어 처리와 관련된 다양한 기능을 제공한다.
- 주요 기능
    - **Word Embeddings**
        - word2vec, fastext, doc2vec 등 다양한 word embedding 모델을 제공
    - **토픽 모델링 (Topic Modeling)**
        - LDA등 문장의 주제를 파악하는 모델 제공
    - **텍스트/word 유사도 계산**
    - **문서 군집화**
        - 비슷한 주제의 문서들을 군집화.
    - 다양한 dataset과 pretrained model 제공
        - https://github.com/piskvorky/gensim-data
- https://radimrehurek.com/gensim/

In [1]:
%pip install gensim

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ------------- -------------------------- 7.9/24.0 MB 37.4 MB/s eta 0:00:01
   ------------------------------ --------- 18.4/24.0 MB 44.6 MB/s eta 0:00:01
   ---------------------------------------- 24.0/24.0 MB 42.3 MB/s eta 0:00:00
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
   ---------------------------------------- 0.0/45.9 MB ? eta -:--:--
   --------- ------------------------------ 11.0/45.9 MB 52.9 MB/s eta 0:00:01
   ------------------- -------------------- 22.8/45.9 MB 55.5 MB/s eta 0:00:01
   ----------------------------- ---------- 33.3/45.9 MB 54.1 MB/s eta 0:00:01
   -------------------------------------- - 44.3/45.9 MB 54.2 MB/s eta 0:00:01
   ---------------------------------------- 45.9/45.9 MB 48.7 MB/s eta 0:00:00

   ---------------------------------------- 0/5 [wrapt]
  Attempting uninstall: numpy
   ------

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.


## 설치
- `pip install gensim`

# Word2Vec 학습

- gensim.models.Word2Vec
- 주요 파라미터
    - sentences
        -  학습에 사용할 문서의 리스트. 각 문서의 단어들을 리스트로 묶고 그 문서들을 리스트로 묶은 중첩 리스트.
        - 예시: \[\['word1', 'word2', 'word3'], \['word4', 'word5']]
    - vector_size
        -  embedding vector 크기. 기본값: 100
    - window
        -  context window 크기. 중심단어를 기준으로 좌우 몇개의 단어를 확인하는지 크기. 기본값: 5
    - min_count
        - 이 설정보다 낮은 빈도로 등장하는 단어는 무시한다. 데이터 노이즈를 줄이는데 도움이된다. 기본값: 5
    - sg
        - 모델 아키텍처 결정.
        - `0`: CBOW, `1`: Skip-gram. 기본값: 0
    - epochs
        - epochs 수 설정. 기본값: 5
    - alpha
        - initial leaning rate. 기본값: 0.025
    - min_alpha
        - 최소 learning rate. 기본값: 0.0001
        - epoch 마다 learning rate를 alpha 에서 min_alpha 까지 선형적으로 줄여나간다.
    - workers
        -  사용 Thread 수. 기본값: 3

## 학습(Train)
1. Word2Vec 의 initializer에 sentences를 넣어 한번에 학습한다.
2. Word2Vec 클래스에 학습 설정을 하고 `train()` 메소드를 이용해 학습한다.
    - epoch 단위로 작업을 할 경우 사용

In [2]:
# 샘플 텍스트 데이터
sentences = [
    "Natural language processing is an exciting field of study",
    "Word embeddings are a type of word representation",
    "Gensim is a powerful library for text processing",
    "Word2Vec creates vector representations of words", 
    "Gensim runs on Linux, Windows and OS X, as well as any other platform that supports Python and NumPy."
    "All Gensim source code is hosted on Github under the GNU LGPL license, maintained by its open source community.",
    "For commercial arrangements, see Business Support.",
    "Gensim can process arbitrarily large corpora, using data-streamed algorithms.",
    "There are no \"dataset must fit in RAM\" limitations."
]

In [ ]:
import re
import nltk
def tokenizer(docs):
    #소문자로 모두 변환
    #알파벳, 숫자, _를 제외한 모든 문자들을 제거
    #단어(어절)단위 토큰화
    return [nltk.word_tokenize(re.sub(r"[^\w\s]","",doc.lower())) for doc in docs]

#[^\w] - \w\s(공백문자) 를 제외한 나머지(^)들을 찾아라.

In [12]:
# [nltk.word_tokenize(re.sub(r"[^\w\s]","",doc.lower())) for doc in sentences]
tokens = tokenizer(sentences)

In [ ]:
import os
os.cpu_count() #현재 병렬로 처리할 수 있는 Thread개수

16

In [ ]:
import os
#train
# Word2Vec 객체를 생성 : 학습데이터, epoch -> 객체 생성할때 모델을 학습. 학습결과 모델을 반환환
from gensim.models import Word2Vec
model1 = Word2Vec(
    sentences=tokens,      # 학습 시킬 데이터
    vector_size=10,        # embedding vector의 차원(한개 단어)
    window=2,              # window size 설정, 주변단어의 개수
    min_count=1, 
    epochs=10,
    workers=os.cpu_count() # 병렬처리 개수
)

In [20]:
# epoch단위 loss등을 확인하고 싶을 때 방법
# 학습 데이터화 epoch은 지정하지 않음 => 학습안된 모델을 반환
model2 = Word2Vec(vector_size=10,window=2,min_count=2,workers=os.cpu_count())
# model에 vocab설정
model2.build_vocab(tokens)
# 학습
epoch = 10
for e in range(epoch):
    model2.train(
        tokens, #학습데이터
        total_examples=model2.corpus_count, #학습데이터(문서) 개수
        epochs=1,
        compute_loss=True,
    )
    loss = model2.get_latest_training_loss()
    print(f'{e} epoch loss: {loss}')

0 epoch loss: 8.462631225585938
1 epoch loss: 19.752229690551758
2 epoch loss: 8.462631225585938
3 epoch loss: 19.77050018310547
4 epoch loss: 11.28350830078125
5 epoch loss: 18.408781051635742
6 epoch loss: 27.582721710205078
7 epoch loss: 12.006998062133789
8 epoch loss: 15.55136489868164
9 epoch loss: 4.231315612792969


## 학습 후 결과 조회

- **KeyedVectors 조회**
    - KeyedVectors는 단어와 vector를 매핑한 객체로 embedding vector를 이용한 다양한 조회를 지원한다.
    - model.wv 로 조회해서 사용.
- **Embedding Vector 조회**
  - model.wv.vectors
- **단어 목록 조회**
    - model.wv.index_to_key, model.wv.key_to_index
- **단어 벡터 조회**
    - model.wv[word]: 특정 단어의 vector반환
- **Vocab에 대상 단어가 있는지 확인**
    - "대상단어" in model.wv
- **유사단어들 찾기**
    - model.wv.most_similar(word)
- **단어간 유사도 비교**
    - model.wv.similarity(word1, word2)
- 유사도를 계산할 때 **코사인 유사도(Cosine Similarity)** 를 사용한다.

> # 코사인 유사도
> - 두 벡터 간의 유사성을 측정하는 중요한 방법 중 하나.
> - 코사인 유사도는 두 벡터 간의 코사인 각도를 이용하여 유사도를 계산한다. 이때 벡터의 **크기는 결과에 영향을 미치지 않고, 오직 방향만이 중요**하다.
> ## 공식
> 
> $$ similarity = cos(\theta) = \frac{A⋅B}{||A||\ ||B||} = \frac{\sum_{i=1}^{n}{A_i×B_i}}{\sqrt{\sum_{i=1}^{n}(A_i)^2}×\sqrt{\sum_{i=1}^{n}(B_i)^2}} $$
> 
> ## 결과 해석
> 
> - **값의 범위**: -1에서 1 사이의 값을 가집니다
>   - 1: 두 벡터가 완전히 동일한 방향 (0도의 cosine 값)
>   - 0: 두 벡터가 직교 (90도의 cosine 값)
>   - -1: 두 벡터가 정반대 방향 (180도의 cosine 값)
> 
> ![cosine_similarity](figures/gensim_consin_sim.png)
>
> ## Python 코사인 유사도 계산
> ```python
> from numpy import dot
> from numpy.linalg import norm
> 
> def cosine_similarity(A, B):
>     return dot(A, B)/(norm(A)*norm(B))
> ```



In [ ]:
model1.wv # KeyedVectors -> 토큰-embedding vector

In [22]:
model1.wv.index_to_key # vocab : token_id -> token(단어)

['gensim',
 'is',
 'of',
 'for',
 'processing',
 'source',
 'as',
 'on',
 'word',
 'are',
 'a',
 'and',
 'words',
 'runs',
 'linux',
 'os',
 'windows',
 'vector',
 'x',
 'well',
 'any',
 'representations',
 'limitations',
 'creates',
 'word2vec',
 'text',
 'platform',
 'library',
 'powerful',
 'representation',
 'type',
 'embeddings',
 'study',
 'field',
 'exciting',
 'an',
 'language',
 'other',
 'that',
 'ram',
 'see',
 'support',
 'can',
 'process',
 'arbitrarily',
 'large',
 'corpora',
 'using',
 'datastreamed',
 'algorithms',
 'there',
 'no',
 'dataset',
 'must',
 'fit',
 'in',
 'business',
 'arrangements',
 'supports',
 'commercial',
 'python',
 'numpyall',
 'code',
 'hosted',
 'github',
 'under',
 'the',
 'gnu',
 'lgpl',
 'license',
 'maintained',
 'by',
 'its',
 'open',
 'community',
 'natural']

In [23]:
model1.wv.key_to_index

{'gensim': 0,
 'is': 1,
 'of': 2,
 'for': 3,
 'processing': 4,
 'source': 5,
 'as': 6,
 'on': 7,
 'word': 8,
 'are': 9,
 'a': 10,
 'and': 11,
 'words': 12,
 'runs': 13,
 'linux': 14,
 'os': 15,
 'windows': 16,
 'vector': 17,
 'x': 18,
 'well': 19,
 'any': 20,
 'representations': 21,
 'limitations': 22,
 'creates': 23,
 'word2vec': 24,
 'text': 25,
 'platform': 26,
 'library': 27,
 'powerful': 28,
 'representation': 29,
 'type': 30,
 'embeddings': 31,
 'study': 32,
 'field': 33,
 'exciting': 34,
 'an': 35,
 'language': 36,
 'other': 37,
 'that': 38,
 'ram': 39,
 'see': 40,
 'support': 41,
 'can': 42,
 'process': 43,
 'arbitrarily': 44,
 'large': 45,
 'corpora': 46,
 'using': 47,
 'datastreamed': 48,
 'algorithms': 49,
 'there': 50,
 'no': 51,
 'dataset': 52,
 'must': 53,
 'fit': 54,
 'in': 55,
 'business': 56,
 'arrangements': 57,
 'supports': 58,
 'commercial': 59,
 'python': 60,
 'numpyall': 61,
 'code': 62,
 'hosted': 63,
 'github': 64,
 'under': 65,
 'the': 66,
 'gnu': 67,
 'lgpl': 

In [24]:
# 특정 단어의 embedding vector 조회
model1.wv['gensim']

array([-0.00530078,  0.00215998,  0.0511257 ,  0.08994552, -0.0929675 ,
       -0.07124555,  0.06510045,  0.08978292, -0.05092729, -0.0380263 ],
      dtype=float32)

In [26]:
'gensim' in model1.wv

True

In [30]:
word = 'numpyall'
if word in model1.wv:
    print(model1.wv[word])
else:
    print('없는 단어')

[ 0.05893448  0.01508251 -0.00718124  0.09339169 -0.04917641 -0.00835453
  0.09189787  0.06754155  0.01489585 -0.08890103]


In [ ]:
# 유사도 검사
model1.wv.most_similar("gensim") #gensim과 가장 유사한 단어를 순서대로 10개 반환
# 단어 유사도 -1 ~ 1

[('numpyall', 0.7189714908599854),
 ('study', 0.713896632194519),
 ('that', 0.6710396409034729),
 ('can', 0.641579270362854),
 ('datastreamed', 0.5988761782646179),
 ('is', 0.5452057123184204),
 ('other', 0.5355265140533447),
 ('maintained', 0.5195193886756897),
 ('representations', 0.5130788087844849),
 ('in', 0.5079874396324158)]

In [ ]:
model1.wv.most_similar("gensim", topn=3)

[('numpyall', 0.7189714908599854),
 ('study', 0.713896632194519),
 ('that', 0.6710396409034729)]

In [37]:
#단어간의 유사도
model1.wv.similarity("gensim","study")
# model1.wv.similarity("python","numpy")

0.7138967

## 모델 저장 및 로딩

### 모델 저장, 로딩
- `model.save('저장파일 경로')`
  - gensim 자체 포맷으로 저장된다.
- `gensim.models.Word2Vec.load('저장파일 경로')`
  - `model.save()`로 저장된 모델을 Loading한다.

In [38]:
import os
os.makedirs('saved_models', exist_ok=True)
model1.save('saved_models/w2v_model.model')

In [39]:
from gensim.models import Word2Vec
load_model = Word2Vec.load('saved_models/w2v_model.model')

In [40]:
load_model.wv.most_similar('gensim')

[('numpyall', 0.7189714908599854),
 ('study', 0.713896632194519),
 ('that', 0.6710396409034729),
 ('can', 0.641579270362854),
 ('datastreamed', 0.5988761782646179),
 ('is', 0.5452057123184204),
 ('other', 0.5355265140533447),
 ('maintained', 0.5195193886756897),
 ('representations', 0.5130788087844849),
 ('in', 0.5079874396324158)]

### Word Embedding Vector만 저장 및 로드
- `KeyedVectors` 를 이용해 저장한다.
    - `model.wv.save_word2vec_format('저장경로', binary=True|False)`
        - binary=True: binary 파일로 저장한다. 용량이 작은 대신 내용확인이 안된다.
        - binary=False: csv(공백구분자) 형식 text로 저장한다. 내용을 확인할 수있지만 파일용량이 크다.
- `KeyedVectors.load_word2vec_format("저장경로", binary=True|False)`
    - 저장시 binary에 맞춰 읽는다.

In [48]:
load_model.wv.save_word2vec_format('saved_models/w2v_model.csv', binary=False)
load_model.wv.save_word2vec_format('saved_models/w2v_model.bin', binary=True)

In [50]:
from gensim.models import KeyedVectors
load_wv_bin = KeyedVectors.load_word2vec_format('saved_models/w2v_model.bin', binary=True)
load_wv_bin.most_similar('gensim')
load_wv_cvs = KeyedVectors.load_word2vec_format('saved_models/w2v_model.csv', binary=False)
load_wv_cvs.most_similar('gensim')

[('numpyall', 0.7189714908599854),
 ('study', 0.713896632194519),
 ('that', 0.6710396409034729),
 ('can', 0.641579270362854),
 ('datastreamed', 0.5988761782646179),
 ('is', 0.5452057123184204),
 ('other', 0.5355265140533447),
 ('maintained', 0.5195193886756897),
 ('representations', 0.5130788087844849),
 ('in', 0.5079874396324158)]

## Pretrained 모델 사용하기
- https://github.com/Kyubyong/wordvectors

In [ ]:
from zipfile import ZipFile
with ZipFile('saved_models/ko_new.zip') as zf:
    zf.extractall('saved_models/ko_new.txt')


In [53]:
ko_wv = KeyedVectors.load_word2vec_format(
    "saved_models/ko_new.txt",
    binary=False,
    encoding='UTF-8'
)

In [58]:
word = '저녁'
word = '호랑이'
result = ko_wv.most_similar(word, topn=5)
result

[('코끼리', 0.7047041654586792),
 ('표범', 0.699328601360321),
 ('멧돼지', 0.6985809803009033),
 ('토끼', 0.6944129467010498),
 ('거북', 0.6699988842010498)]

In [59]:
ko_wv[word]

array([ 1.3842834 , -0.356262  ,  0.55099374, -0.12755577, -0.8897868 ,
       -0.01085023,  0.4051852 , -0.00849318,  0.7922764 ,  0.61044914,
       -0.21740127,  0.7326123 ,  0.00975952,  1.2224952 ,  1.7374074 ,
        0.550102  , -0.04433551,  0.46078685,  0.7088423 , -0.12447739,
        0.26103678, -1.7141577 ,  1.1090391 , -0.06266715,  0.5393867 ,
       -0.5670158 ,  2.8203938 ,  1.4095887 ,  0.25168103, -0.36968148,
        0.23580536,  0.0387576 , -0.11112779, -0.6910354 ,  1.0538206 ,
       -1.4043067 ,  0.25224012,  2.213298  , -0.8007955 ,  0.09763678,
       -0.6443904 ,  0.8280104 , -0.8043615 , -2.3739452 , -0.1739332 ,
       -0.8374608 ,  0.48010096,  1.5835998 , -1.7894523 ,  1.0660172 ,
        0.9587638 ,  0.20674476,  0.53526455,  0.16311654, -1.6024637 ,
       -0.39326617, -0.903832  , -0.63576716,  0.10457885, -0.9901898 ,
        0.44487828, -0.01300097, -0.6176418 , -0.94335663,  0.8662551 ,
       -0.7614388 ,  0.42271316, -1.0146952 , -0.12537627, -0.02

# 빅카인즈 뉴스 데이터를 이용한 Word2Vec 학습
- 빅카인즈
    - 한국언론진흥재단에서 운영하는 뉴스빅데이터 분석 서비스 사이트
- 빅카인즈에서 특정 분야의 기사들을 수집해서 학습시킨다.
    - https://www.bigkinds.or.kr/
    - 회원가입 (구글, 네이버, 카카오 계정으로 가입 가능) 후 로그인
    - 뉴스분석 > 뉴스검색$\cdot$분석 클릭
    ![word2vec_bigkinds1.png](figures/word2vec_bigkinds1.png)
    - 기간, 언론사, 분류, 상세검색 등 검색 조건입력 후 조회
    ![word2vec_bigkinds1.png](figures/word2vec_bigkinds2.png)
    - 결과 다운로드
        - step3 분석결과및 시각화 -> 맨 아래 `엑셀다운로드` 클릭 
    ![word2vec_bigkinds1.png](figures/word2vec_bigkinds3.png)

In [3]:
import pandas as pd
df = pd.read_excel('NewsResult_20250223-20250523.xlsx')
df.head()

c:\Users\Playdata\AppData\Local\miniconda3\envs\dl\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부
0,1.101001e+06,20250523,한겨레,김남일 기자,검사 판사가 유권자 결정 바꾸는 선거법 ‘삭제→부활→삭제’ 30년사,정치>국회_정당,사회>사건_사고,정치>선거,범죄>기업범죄>거래제한,재해>자연재해>홍수,NaN,"김,3천만원,김문수,김지영,김용빈,유상범,이재명,마타,신중한","영국,공직선거 및 선거부정방지법","캐나다,대검찰청,민주당,국회 법사위,헌법연구관,검찰,국회 법제사법위,경찰,더불어민주...","검사,판사,유권자,결정,선거법,삭제,부활,삭제,30년,혐의,공직선거법,위반,재판,이...","후보자,공직선거법,허위사실공표죄,선거법,이재명,허위사실공표,대법원,유권자,위원회,통...",공직선거법 위반 혐의로 재판을 받는 이재명 더불어민주당 대통령 후보는 대법원 전원합...,http://www.hani.co.kr/arti/politics/election/1...,NaN
1,1.100701e+06,20250523,세계일보,김정모,환경장관 만난 김태흠 충남지사 “청양 부여 지천댐 조속 건설을”,사회>사회일반,지역>강원,지역>충남,재해>자연재해>가뭄,재해>자연재해>홍수,NaN,"김완섭,김태흠","정부세종청사,은산면,수원,부여군,태평양,아시아,청양군,장평면,용수,보령댐,대청댐,부...","환경부,정부,부여군,환경부장관,충남지사,충남,보령댐,용수,청양","환경장관,청양,김태흠,충남,지사,부여,건설,지천댐,조속,김태흠,충남,지사,청양,부여...","지천댐,청양,충남,부여,보령댐,후보지,부여군,대청댐,수원,김태흠,환경부,아시아,김완...",김태흠 충남지사가 청양 부여 지천댐 건설을 조속하게 추진해 줄 것을 정부에 요청했다...,http://www.segye.com/content/html/2025/05/22/2...,NaN
2,1.100801e+06,20250523,조선일보,유석재 기자,[유석재의 돌발史전] 나는 이제 더 이상 ‘마오’를 존경하지 않는다,국제>중국,문화>출판,국제>일본,사고>산업사고>화재,재해>자연재해>홍수,사회>사회갈등>테러행위,"노무현,마오쩌둥,장융,프랑크,리영희,마오,리즈수이,왕단,존 핼리데이,류샤오치,한상일...","캐나다,서울,중국현대사,서구,마오,한국,공산,독창,동아시아,대약진운동,내몽골,대전,...","중국 문화대혁명,일화,동원,조선일보,IMF,중국,현대,마오,현대사학자,대동,대약진운...","마오,존경,전환시대,논리,50주년,文革,일반적,신문기사,돌발,인터넷상,연재,뉴스레터...","중국,문혁,마오,리영희,모택동,마오쩌둥,사람들,전환시대,왕단,지옥도,서울,현대사,송...","일반적인 신문기사라면야 그럴 일이 거의 없겠습니다만, 이 ‘돌발史전’은 인터넷상에만...",https://www.chosun.com/culture-life/relion-aca...,NaN
3,1.100611e+06,20250523,서울신문,세종 이현정 기자·부처종합\n이현정,“이 정책만은 정권 초월해야” 부처별 ‘지속 과제’ 지키기 안간힘,사회>사회일반,정치>청와대,NaN,재해>자연재해>가뭄,재해>자연재해>홍수,NaN,NaN,"초월,농축산","보건복지부,기재부,농식품부,산업부,고용노동부,복지부,사회부,건강보험정책심의위원회,정...","정책,정권,초월,부처,지속,안간힘,정부,출범,분주,정책,정리,대통령,정책,기조,기재...","관계자,업무보고,농식품부,사각지대,농축산물,기재부,중장년,일자리,소득세,경제공약,분주","“누가 대통령 되든 정책 기조 달라져” \n기재부, 대선 후보 경제공약 분석 중 \...",http://go.seoul.co.kr/news/newsView.php?id=202...,NaN
4,1.100751e+06,20250522,아시아투데이,NaN,안전사고 대국 中 잇따라 인명 사고 발생,국제>아시아,국제>중남미,국제>중국,사고>산업사고>붕괴,재해>자연재해>눈사태_산사태,재해>자연재해>폭염,홍순도,"쓰촨,남서,大方)현 창스,창스얼,구이저우,광시(廣西)장족자치구,중국,廣東,완위안,다...","신화(新華)통신,대국,아시아투데이","안전사,대국,발생,인명,사고,아시아투데이,홍순,베이징,특파,안전사,대국,중국,인명,...","중국,베이징,남서부,빈발,궈와,果瓦,長石,新華,廣西,萬源,완위,장족자치구,2명,창스...",아시아투데이 홍순도 베이징 특파원 = 안전사고 대국이 될 수밖에 없는 중국에 최근 ...,https://www.asiatoday.co.kr/view.php?key=20250...,NaN


In [4]:
df = df[['제목','본문']]
df.head()



,제목,본문
0,검사 판사가 유권자 결정 바꾸는 선거법 ‘삭제→부활→삭제’ 30년사,공직선거법 위반 혐의로 재판을 받는 이재명 더불어민주당 대통령 후보는 대법원 전원합...
1,환경장관 만난 김태흠 충남지사 “청양 부여 지천댐 조속 건설을”,김태흠 충남지사가 청양 부여 지천댐 건설을 조속하게 추진해 줄 것을 정부에 요청했다...
2,[유석재의 돌발史전] 나는 이제 더 이상 ‘마오’를 존경하지 않는다,"일반적인 신문기사라면야 그럴 일이 거의 없겠습니다만, 이 ‘돌발史전’은 인터넷상에만..."
3,“이 정책만은 정권 초월해야” 부처별 ‘지속 과제’ 지키기 안간힘,"“누가 대통령 되든 정책 기조 달라져” \n기재부, 대선 후보 경제공약 분석 중 \..."
4,안전사고 대국 中 잇따라 인명 사고 발생,아시아투데이 홍순도 베이징 특파원 = 안전사고 대국이 될 수밖에 없는 중국에 최근 ...


제목                  환경장관 만난 김태흠 충남지사 “청양 부여 지천댐 조속 건설을”
본문    김태흠 충남지사가 청양 부여 지천댐 건설을 조속하게 추진해 줄 것을 정부에 요청했다...
Name: 1, dtype: object

In [ ]:
sentences = []
for i in range(len(df)):
    sentences.append(df.iloc[i][0])
    sentences.append(df.iloc[i][1])
sentences

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

C:\Users\Playdata\AppData\Local\Temp\ipykernel_15380\2887423665.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentences.append(df.iloc[i][0])
C:\Users\Playdata\AppData\Local\Temp\ipykernel_15380\2887423665.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentences.append(df.iloc[i][1])


['검사 판사가 유권자 결정 바꾸는 선거법 ‘삭제→부활→삭제’ 30년사',
 '공직선거법 위반 혐의로 재판을 받는 이재명 더불어민주당 대통령 후보는 대법원 전원합의체의 유죄 취지 파기환송에 따라 사실상 형량만 정하면 되는 상태다. 대통령에 당선되면 이 재판이 중단되는지, 유죄가 확정되면 대통령직을 유지할 수 있는지를 두고 논란이 인다. 민주당은 이 논란을 근원적으로 해결하기 위해 검찰이 이 후보 기소 근거로 삼은 공직선거법 제25..',
 '환경장관 만난 김태흠 충남지사 “청양 부여 지천댐 조속 건설을”',
 '김태흠 충남지사가 청양 부여 지천댐 건설을 조속하게 추진해 줄 것을 정부에 요청했다. \n \n 김 지사는 22일 정부세종청사에서 김완섭 환경부장관을 만나 목마른 충남의 갈증 해소와 반복되는 홍수 피해 방지 등 기후대응댐 조성 신속 추진의 필요성을 설명했다. \n \n 지천댐은 청양군 장평면과 부여군 은산면 일원에 저수용량 5900만㎥ 규모로 건립한다는 계획이..',
 '[유석재의 돌발史전] 나는 이제 더 이상 ‘마오’를 존경하지 않는다',
 '일반적인 신문기사라면야 그럴 일이 거의 없겠습니다만, 이 ‘돌발史전’은 인터넷상에만 연재되는 뉴스레터 기사이기 때문에 종종 제 개인적인 얘기가 등장할 때가 있습니다. \n \n그런데 지난주 뽀빠이 이상용이 1980년에 쓴 유머 서적을 소개하던 중 마오쩌둥(毛澤東)에 관련된 농담이 나오는 부분에서, 저는 과거에 이 농담을 떠올리고 불편한 적이 있었다는 언급을..',
 '“이 정책만은 정권 초월해야” 부처별 ‘지속 과제’ 지키기 안간힘',
 '“누가 대통령 되든 정책 기조 달라져” \n기재부, 대선 후보 경제공약 분석 중 \n산업부 ‘에너지 분야’ 일관성 강조 \n농식품부 ‘기후 대응’ 핵심과제 지정\n2주 앞으로 다가온 새 정부 출범을 앞두고 각 부처는 ‘정권을 초월해 지속돼야 할 정책’ 정리에 분주하다. 아직 차기 정부의 업무보고 콘셉트, 시기, 형식 모두 불투명하지만 부처들은 핵심 과제를 가다..',
 '안전사

In [19]:

import re
import nltk
def tokenizer(docs):
    #소문자로 모두 변환
    #알파벳, 숫자, _를 제외한 모든 문자들을 제거
    #단어(어절)단위 토큰화
    return [nltk.word_tokenize(re.sub(r"[^\w\s]","",doc.lower())) for doc in docs]


In [20]:
tokens =tokenizer(sentences)

In [22]:
import os
from gensim.models import Word2Vec
model1 = Word2Vec(
    sentences=tokens,      # 학습 시킬 데이터
    vector_size=10,        # embedding vector의 차원(한개 단어)
    window=2,              # window size 설정, 주변단어의 개수
    min_count=1, 
    epochs=10,
    workers=os.cpu_count() # 병렬처리 개수
)


In [27]:
model1.wv.key_to_index


{'미얀마': 0,
 '등': 1,
 '산불': 2,
 '피해': 3,
 '있다': 4,
 '밝혔다': 5,
 '지진': 6,
 '위해': 7,
 '따르면': 8,
 '규모': 9,
 '경북': 10,
 '지난': 11,
 '발생한': 12,
 '수': 13,
 '있는': 14,
 '이번': 15,
 '서울': 16,
 '이날': 17,
 '것으로': 18,
 '이': 19,
 '한': 20,
 '피해를': 21,
 '강진': 22,
 '발생': 23,
 '기자': 24,
 '위한': 25,
 '지역': 26,
 '지진이': 27,
 '28일': 28,
 '미국': 29,
 '오전': 30,
 '재난': 31,
 '여름철': 32,
 '지난달': 33,
 '가운데': 34,
 '및': 35,
 '아시아투데이': 36,
 '오후': 37,
 '인한': 38,
 '지원': 39,
 '최근': 40,
 '통해': 41,
 '중': 42,
 '대형': 43,
 '대응': 44,
 '태국': 45,
 '올해': 46,
 '전': 47,
 '큰': 48,
 '대한': 49,
 '사망자': 50,
 '강진으로': 51,
 '일본': 52,
 '산사태': 53,
 '더': 54,
 '복구': 55,
 '지난해': 56,
 '산불이': 57,
 '2025년': 58,
 '중국': 59,
 '77의': 60,
 '등에': 61,
 '산불로': 62,
 '피해가': 63,
 '전국': 64,
 '약': 65,
 '폭염': 66,
 '발생했다': 67,
 '최대': 68,
 '이후': 69,
 '3월': 70,
 '인천': 71,
 '함께': 72,
 '가장': 73,
 '주민': 74,
 '77': 75,
 '긴급': 76,
 '13일': 77,
 '경기': 78,
 '경남': 79,
 '등을': 80,
 '아침': 81,
 '미세먼지': 82,
 '지진으로': 83,
 '기후': 84,
 '포항': 85,
 '입은': 86,
 '정부': 87,
 '대비': 88,
 '강진이':

In [28]:
model1.wv.most_similar("기자")

[('덮치면서', 0.9822012782096863),
 ('드릴로', 0.970028817653656),
 ('지반침하', 0.9665538668632507),
 ('오로라', 0.9640517830848694),
 ('안전점검을', 0.9636015295982361),
 ('가디언은', 0.9587206840515137),
 ('평균', 0.9583089351654053),
 ('자체가', 0.9571895599365234),
 ('생태계를', 0.9570465087890625),
 ('중심', 0.9567498564720154)]

In [ ]:
# 문서(doc): 개별 뉴스기사 (제목 + 본문)
# 전처리 & 토큰화 - 소문자 통일. 형태소단위 토큰화.
# 
# gensim을 이용해서 embedding 모델 학습.
#
# 결과 확인 -> 유사도 검사. 

In [34]:
#GPT
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tqdm import tqdm

nltk.download("punkt")  # 토크나이저 사용할 때 필요

# 🔹 새 전처리 함수: 영어 텍스트 전처리 기준
def tokenizer(docs):
    return [
        word_tokenize(re.sub(r"[^\w\s]", "", doc.lower()))
        for doc in docs
    ]

# 제목 + 본문 결합
docs = df["제목"] + " " + df["본문"]
corpus = tokenizer(docs)

# 🔹 Word2Vec 모델 학습
model = Word2Vec(
    sentences=corpus,
    vector_size=100,
    window=5,
    min_count=1,
    workers=4,
    sg=1  # skip-gram
)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Playdata\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [35]:
model.wv.key_to_index

{'미얀마': 0,
 '등': 1,
 '산불': 2,
 '피해': 3,
 '있다': 4,
 '밝혔다': 5,
 '지진': 6,
 '위해': 7,
 '따르면': 8,
 '규모': 9,
 '경북': 10,
 '지난': 11,
 '발생한': 12,
 '수': 13,
 '있는': 14,
 '이번': 15,
 '서울': 16,
 '이날': 17,
 '것으로': 18,
 '이': 19,
 '한': 20,
 '피해를': 21,
 '강진': 22,
 '발생': 23,
 '기자': 24,
 '위한': 25,
 '지역': 26,
 '지진이': 27,
 '28일': 28,
 '미국': 29,
 '오전': 30,
 '재난': 31,
 '여름철': 32,
 '지난달': 33,
 '가운데': 34,
 '및': 35,
 '아시아투데이': 36,
 '오후': 37,
 '인한': 38,
 '지원': 39,
 '최근': 40,
 '통해': 41,
 '중': 42,
 '대형': 43,
 '대응': 44,
 '태국': 45,
 '올해': 46,
 '전': 47,
 '큰': 48,
 '대한': 49,
 '사망자': 50,
 '강진으로': 51,
 '일본': 52,
 '산사태': 53,
 '더': 54,
 '복구': 55,
 '지난해': 56,
 '산불이': 57,
 '2025년': 58,
 '중국': 59,
 '77의': 60,
 '등에': 61,
 '산불로': 62,
 '피해가': 63,
 '전국': 64,
 '약': 65,
 '폭염': 66,
 '발생했다': 67,
 '최대': 68,
 '이후': 69,
 '3월': 70,
 '인천': 71,
 '함께': 72,
 '가장': 73,
 '주민': 74,
 '77': 75,
 '긴급': 76,
 '13일': 77,
 '경기': 78,
 '경남': 79,
 '등을': 80,
 '아침': 81,
 '미세먼지': 82,
 '지진으로': 83,
 '기후': 84,
 '포항': 85,
 '입은': 86,
 '정부': 87,
 '대비': 88,
 '강진이':

In [36]:

# 🔹 예시 단어 유사도
print("\n[유사 단어]")
word = "미얀마"
if word in model.wv:
    print(model.wv.most_similar(word))
else:
    print(f"'{word}' 단어는 vocabulary에 없음.")



[유사 단어]
[('사망자', 0.9791519641876221), ('강진', 0.9726977348327637), ('수가', 0.9686023592948914), ('사망', 0.9669117331504822), ('태국', 0.9662778377532959), ('늘어', 0.9642844796180725), ('이틀째인', 0.96379154920578), ('군부', 0.9631881713867188), ('사망자가', 0.9630048274993896), ('군정', 0.9613040685653687)]


In [37]:

# 🔹 4. 단어 유사도 확인
print("\n[단어 유사도]")
print("미얀마 vs 태국:", model.wv.similarity("미얀마", "태국"))



[단어 유사도]
미얀마 vs 태국: 0.9662779
